In [1]:
## General librairies
import pandas as pd
import numpy as np
import _pickle as pickle
import matplotlib.pyplot as plt
import random
import sys
import time
from copy import deepcopy
import os

from keras.utils import np_utils
from sklearn.model_selection import train_test_split


import dateutil.parser

Using TensorFlow backend.


In [2]:
specs = 'specs.csv'
test = 'test.csv'
train = 'train.csv'
train_labels = 'train_labels.csv'
subs = 'sample_submission_exemple.csv'

In [3]:
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

def relative_time(x):
    x1 = []
    for elt in x:
        x1.append((elt-x[0]).item()/1000000000)
    return x1

def categorise(x):
    dico = {}
    count = 0
    for elt in x:
        if not(elt in dico):
            dico[elt] = count
            count += 1
    return dico

def padding( dataset, n):
    d = list(np.zeros(len(dataset)))
    c = 0
    count = 0
    for elt in dataset:
        
        if count % 100 == 0:
            print(count)
        u = elt.shape[0]
        
        if u<200:
            c += 1
        
        if u > n:
            d[count] = elt[-n:]
        else:
            a = np.zeros(((n-u), elt.shape[1]))-1
            elt = np.concatenate([a, elt], axis = 0)
            d[count] = elt
        count += 1
    return d

def build_line(df, installation_id, game_session):
    
    df1 = df[(df['installation_id']==installation_id)&(df['date']<=df[(df['installation_id']==installation_id)&(df['game_session']==game_session)]['date'].iloc[0])]
    dico = categorise(df1['game_session'].unique())
    df1 = df1.replace({'game_session' : dico})
    x = relative_time(df1['date'].values)
    
    df1['time_delta'] = x
    
#     df1 = df1.drop_duplicates(subset = ['event_id'])
    
    cats_to_keep = ['game_session','time_delta','title','type', 'world',  'event_count', 'game_time', 'event_code']
    df1 = df1[cats_to_keep]
    return df1

def format_input(X):
    return [X[:,:,[0,1,5,6]], X[:,:,2].reshape((X.shape[0], X.shape[1],1)), X[:,:,3].reshape((X.shape[0], X.shape[1],1)), X[:,:,4].reshape((X.shape[0], X.shape[1],1)),X[:,:,7].reshape((X.shape[0], X.shape[1],1))]


#### Loading Data

In [ ]:
df = pd.read_csv(train)
labels = pd.read_csv(train_labels)
dtitle, dtype, dworld, devent = load('dicos')

In [ ]:
labels = pd.read_csv(train_labels)

In [ ]:
df.head()

#### Adding some formating to dataframe

In [ ]:
df = df[(df['event_code']<4020)|(df['event_code']>4080)]

In [ ]:
df.shape

In [ ]:
ids = df[df['type']=='Assessment']['installation_id'].unique()
df = df[df.installation_id.isin(ids)]
df = df[df.installation_id.isin(labels.installation_id.unique())]

dtitle, dtype, dworld, devent = load('dicos')

dtitle = categorise(df['title'])
dtype = categorise(df['type'])
dworld = categorise(df['world'])
devent = categorise(df['event_code'])

save((dtitle, dtype, dworld, devent), 'dico')

df = df.replace({'title' : dtitle})
df = df.replace({'type' : dtype})
df = df.replace({'world' : dworld})
df = df.replace({'event_code' : devent})

# df['date'] = df['timestamp'].apply(dateutil.parser.parse)
# df = df.sort_values(by = ['date'], ascending =True)
# df = df.reset_index()

# df = df.reset_index()

In [ ]:
df.shape

In [ ]:
from multiprocess import Pool
def f(x):
    import dateutil.parser
    return dateutil.parser.parse(x)

if __name__ == '__main__':
    p = Pool(8)
    vect = p.map(f, df['timestamp'].values)

df['date'] = vect
df = df.sort_values(by = ['date'], ascending =True)
df = df.reset_index()

df = df.reset_index()

In [ ]:
save(df, 'time train')

In [ ]:
df = load('time train')

In [ ]:
df = df.sort_values(by = ['date'], ascending =True)
df = df.sort_values(by = ['installation_id'], ascending =True)

dico_id = {}

vect = df['installation_id'].values
count = 0
for elt in vect:    
    if count == 0:
        val1 = 0
        mem = elt
    
    if elt != mem:
        val2 = count
        dico_id[mem] = [val1, val2]
        val1 = count
    
    if count == vect.shape[0]-1:
        dico_id[mem] = [val1, vect.shape[0]]
    
    mem = elt
    count += 1

In [ ]:
per_id = list(np.zeros(3614))

count = 0
for elt in labels['installation_id'].unique():
    per_id[count] = labels[labels['installation_id']==elt]
    count += 1



dataset = list(np.zeros(labels['installation_id'].unique().shape[0]))

count = 0

for i in range(len(per_id)):
    print(i)
    temp = list(np.zeros(per_id[i].shape[0]))
    rows = dico_id[per_id[i].iloc[0]['installation_id']]
    df1 = df.iloc[rows[0]:rows[1]]
    df1 = df1.sort_values(by = ['date'], ascending =True)
#     df1 = df1.drop_duplicates(subset = ['event_id'])
#     print(df1)
#     df1 = df[df['installation_id'] == per_id[i].iloc[0]['installation_id']]
#     print(df1.head())
    for j in range(len(temp)):
        installation_id = per_id[i].iloc[j]['installation_id']
        game_session = per_id[i].iloc[j]['game_session']
        df2 = build_line(df1, installation_id, game_session)

#         temp[j] = df2.values
        dataset[i] = df2.values
        count += 1

In [ ]:
save(dataset, 'non_padded')

In [ ]:
dataset[0].shape

In [ ]:
M = 0
for i in dataset:
#     if i.shape[0] > M:
        M += i.shape[0]


In [ ]:
M/17000

In [ ]:
del df
del dataset

In [5]:
dataset = load('non padded')

# pad = 2000

# dataset = padding(dataset,pad)

In [ ]:
labels = load('train_labels')
X = np.array(dataset)
Y = labels['accuracy_group'].values
y = np_utils.to_categorical(Y)

from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y),
                                               Y)
del dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

#15102
# X_train = X[:15102]
# X_test = X[15102:]
# y_train = y[:15102]
# y_test = y[15102:]



X_train = format_input(X_train)
X_test = format_input(X_test)

In [ ]:
del X

In [ ]:
import gc
gc.collect()

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Dropout, Dense,CuDNNLSTM, Flatten, Embedding, TimeDistributed, Concatenate, LSTM, BatchNormalization, Lambda, Reshape
from keras.regularizers import l2

pad_size = pad

num = Input(shape = (pad_size, 4))
title = Input(shape = (pad_size, 1))
type1 = Input(shape = (pad_size, 1))
world = Input(shape = (pad_size, 1))
event = Input(shape = (pad_size, 1))

inputs = [num, title, type1, world, event]

num_emb = TimeDistributed(Dense(16))(num)

title_emb = TimeDistributed(Embedding( 44 ,8))(title)
# title_emb = Lambda(lambda x: x, output_shape = lambda s:s)(title_emb)
title_emb = Reshape((-1,8))(title_emb)

type_emb = TimeDistributed(Embedding( 4 ,2))(type1)
# type_emb = Lambda(lambda x: x, output_shape = lambda s:s)(type_emb)
type_emb = Reshape((-1,2))(type_emb)

world_emb = TimeDistributed(Embedding( 4 ,2))(world)
# world_emb = Lambda(lambda x: x, output_shape = lambda s:s)(world_emb)
world_emb = Reshape((-1,2))(world_emb)

event_emb = TimeDistributed(Embedding( 42 ,8))(event)
# event_emb = Lambda(lambda x: x, output_shape = lambda s:s)(event_emb)
event_emb = Reshape((-1,8))(event_emb)

embedded = [title_emb, type_emb, world_emb, event_emb]

merged = Concatenate()(embedded + [num_emb])

merged = BatchNormalization()(merged)

merged = TimeDistributed(Dense(16))(num)

lstm_out = CuDNNLSTM(64, return_sequences = False)(merged)

# lstm_out = Flatten()(lstm_out)

td = Dropout(0.2)(lstm_out)
td = Dense(32, activation = 'relu')(td)
td = Dropout(0.2)(td)

td = Dense(4, activation = 'sigmoid')(td)

model = Model(inputs , td)

In [ ]:
model.summary()

In [ ]:
import keras
from sklearn.metrics import cohen_kappa_score
import keras.backend as K
from keras.optimizers import SGD

optimizer = SGD(0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

batch_size = 64
epochs = 15

history = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), class_weight=class_weights)

In [ ]:

optimizer = SGD(0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

batch_size = 64
epochs = 10

history = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), class_weight=class_weights)

In [ ]:
optimizer = SGD(0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

batch_size = 64
epochs = 5

history = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), class_weight=class_weights)

In [ ]:
a = model.predict(X_test)

In [ ]:
cohen_kappa_score(np.argmax(y_test, axis = 1),np.argmax(a, axis = 1), weights = 'quadratic')

In [ ]:
save(model, 'model')

In [ ]:
del X_train
del X_test

gc.collect()

## Now on test data

In [ ]:
df = load('test')
ids = df['installation_id'].unique()

In [ ]:
df1 = df[df['installation_id']=='0500e23b']

df1['date'] = df1['timestamp'].apply(dateutil.parser.parse)

In [ ]:
df1 = df1.sort_values(by = ['date'], ascending =True)

In [ ]:

dtitle, dtype, dworld, devent = load('dicos')
df = df.replace({'title' : dtitle})
df = df.replace({'type' : dtype})
df = df.replace({'world' : dworld})
df = df.replace({'event_code' : devent})
# df['date'] = df['timestamp'].apply(dateutil.parser.parse)
# df = df.sort_values(by = ['date'], ascending =True)
# df = df.reset_index()
# df = df.reset_index()

In [ ]:
from multiprocess import Pool
def f(x):
    import dateutil.parser
    return dateutil.parser.parse(x)

if __name__ == '__main__':
    p = Pool(8)
    vect = p.map(f, df['timestamp'].values)
    
df['date'] = vect
df = df.sort_values(by = ['date'], ascending =True)
df = df.reset_index()
df = df.reset_index()

In [ ]:
df = df.sort_values(by = ['date'], ascending =True)
df = df.sort_values(by = ['installation_id'], ascending =True)

dico_id = {}

vect = df['installation_id'].values
count = 0
for elt in vect:    
    if count == 0:
        val1 = 0
        mem = elt
    
    if elt != mem:
        val2 = count
        dico_id[mem] = [val1, val2]
        val1 = count
    
    if count == vect.shape[0]-1:
        dico_id[mem] = [val1, vect.shape[0]]
    
    mem = elt
    count += 1

In [ ]:
dataset = list(np.zeros(ids.shape[0]))


for i in range(ids.shape[0]):
    print(i)
    
    installation_id = ids[i]
    
    rows = dico_id[installation_id]
    df1 = df.iloc[rows[0]: rows[1]]
    df1 = df[df['installation_id'] == installation_id]
    df1 = df1.sort_values(by = ['date'], ascending =True)
    
    game_session = df1.iloc[-1]['game_session']
#     print(df1.iloc[-1]['type'])
    
    df2 = build_line(df1, installation_id, game_session)
        
    dataset[i] = df2.values

# for i in range(len(per_id)):
#     print(i)
#     temp = list(np.zeros(per_id[i].shape[0]))
#     df1 = df[df['installation_id'] == per_id[i].iloc[0]['installation_id']]
# #     print(df1.head())
#     for j in range(len(temp)):
#         installation_id = per_id[i].iloc[j]['installation_id']
#         game_session = per_id[i].iloc[j]['game_session']
#         df2 = build_line(df1, installation_id, game_session)

#         temp[j] = df2.values
#     dataset.extend(temp)

In [ ]:
del df
save(dataset, 'non_padded_test')

In [ ]:
dataset = load('non_padded_test')
dataset = padding(dataset,pad)

In [ ]:
X = np.array(dataset)
X = format_input(X)

In [ ]:
model = load('model')
pred = model.predict(X)


pred = np.argmax(pred, axis = 1)

In [ ]:
ids = ids.reshape((ids.shape[0], 1))
pred = pred.reshape((pred.shape[0], 1))

df1 = pd.DataFrame(np.concatenate([ids, pred], axis = 1), index = ids.reshape(ids.shape[0]), columns = ['installation_id','accuracy_group'])

In [ ]:
pred

In [ ]:
sample_sub = pd.read_csv(subs)
sample_sub['accuracy_group'] = df1.loc[sample_sub['installation_id'].values]['accuracy_group'].values

In [ ]:
sample_sub.to_csv('sample_submission.csv', index = False)